In [1]:
import torch,sys,mujoco,time
import numpy as np
import matplotlib.pyplot as plt
import sys,os
sys.path.append('../utils')
from mujoco_parser import MuJoCoParserClass
from util import rpy2R,get_idxs
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

xml_path = '../asset/smplh_rig/scene_smplh_rig.xml'
env = MuJoCoParserClass(name='Common-Rig',rel_xml_path=xml_path,VERBOSE=False)

for body_name in env.body_names:
    if body_name in ['world']: continue
    for geom_idx in env.get_geom_idxs_from_body_name(body_name=body_name):
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]

MuJoCo version:[3.1.2]


In [2]:
hand_idxs = list(range(30,49)) + list(range(57,76))
hand_joint_names = [env.joint_names[i-6] for i in hand_idxs]

ls = env.get_p_body('left_shoulder')
le = env.get_p_body('left_elbow')
lw = env.get_p_body('left_wrist')
rs = env.get_p_body('right_shoulder')
re = env.get_p_body('right_elbow')
rw = env.get_p_body('right_wrist')

len_ls2le = np.linalg.norm(le-ls)
len_le2lw = np.linalg.norm(lw-le)
len_rs2re = np.linalg.norm(re-rs)
len_re2rw = np.linalg.norm(rw-re)

BODY_NAMES = ['left_elbow','left_wrist','right_elbow','right_wrist']
JOINT_NAMES = ['l_shoulder1', 'l_shoulder2', 'l_shoulder3', 'l_elbow', 'l_wrist1', 'l_wrist2', 'l_wrist3',
               'r_shoulder1', 'r_shoulder2', 'r_shoulder3', 'r_elbow', 'r_wrist1', 'r_wrist2', 'r_wrist3']

p_target = {}
p_target['left_elbow'] = ls + len_ls2le * np.array([0.0,1.0,0.0])
p_target['left_wrist'] = p_target['left_elbow'] + len_le2lw * np.array([0.0,0.0,1.0])
p_target['right_elbow'] = rs + len_rs2re * np.array([0.0,-1.0,0.0])
p_target['right_wrist'] = p_target['right_elbow'] + len_re2rw * np.array([0.0,0.0,1.0])

idxs = get_idxs(env.rev_joint_names,JOINT_NAMES)
q_mins_body,q_maxs_body = env.rev_joint_mins[idxs],env.rev_joint_maxs[idxs]
joint_idxs_fwd_body = env.get_idxs_fwd(joint_names=JOINT_NAMES)
joint_idxs_jac_body = env.get_idxs_jac(joint_names=JOINT_NAMES)

q_body = env.get_qpos_joints(JOINT_NAMES)
body_diff = 1e3
prev_q_pos = env.data.qpos.copy()

while body_diff > 1e-4:
    J_list,ik_err_list = [],[]
    for ik_body_name in BODY_NAMES:
        ik_p_trgt = p_target[ik_body_name]
        ik_R_trgt = None
        IK_P = True
        IK_R = False
        J,ik_err = env.get_ik_ingredients(
            body_name=ik_body_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
            IK_P=IK_P,IK_R=IK_R)
        J_list.append(J)
        ik_err_list.append(ik_err)
    J_stack      = np.vstack(J_list)
    ik_err_stack = np.hstack(ik_err_list)
    dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
    q_body = q_body + dq[joint_idxs_jac_body]

    q_body = np.clip(q_body,q_mins_body,q_maxs_body)
    env.forward(q=q_body,joint_idxs=joint_idxs_fwd_body)

    q_env = env.data.qpos.copy()
    body_diff = np.linalg.norm(q_env - prev_q_pos)
    prev_q_pos=q_env.copy()

q_init = q_body.copy()

In [3]:
SAVE_VIDEO = True

check_number = 494
qpos_data = np.load('../data/both57m_handonly/' + '%d.npy' % check_number)

env.init_viewer(viewer_title='SMPLH-Rig',viewer_width=800,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=180,distance=1.4,elevation=-10,lookat=[0.02,-0.01,1.73])
env.reset()
env.forward(q=q_init,joint_idxs=joint_idxs_fwd_body)

tick = 0
seq_len = qpos_data.shape[0]
video = []

while(env.is_viewer_alive() and tick < seq_len):
    q = qpos_data[tick,hand_idxs]
    
    env.forward(q=q,joint_idxs=hand_idxs)
    env.render()
    env.plot_T(p=np.array([0,0,1.8]),R=np.eye(3,3),PLOT_AXIS=False,label="tick:[%d]"%(tick))
    
    frame = env.grab_image()
    video.append(frame)

    tick = tick + 1
    if tick == seq_len:
        if not SAVE_VIDEO: tick = 0

env.close_viewer() 

In [4]:
import mediapy
mediapy.show_video(video,fps=30)

This browser does not support the video tag.
